## CSE 532 Assignment 4 (Due 4/27/24)

**Note: As with the previous assignment you should submit a separate document (.pdf or .doc(x)) with your responses to the analysis portion of the problems.** 

**1. (Machine Learning (Classification))** <br>a. Choose one of the [toy classification datasets](https://scikit-learn.org/stable/datasets/toy_dataset.html) bundled with sklearn **other than the digits dataset**. <br> b. Train **three** distinct sklearn classification estimators for the chosen dataset and compare the results to see which one performs the best when using **2-fold cross-validation**.  Note that you should use three distinct classification models here (not just tweak underlying parameters).  A relatively complete listing of the available estimators can be found here (https://scikit-learn.org/stable/supervised_learning.html) -- but make sure you only use classifiers!  Unless you have an inclination to do otherwise, I recommend using the model default parameters when available.   <br> c. Repeat a. for **20-fold cross-validation**. Explain in a paragraph the difference in your results when using 20-fold vs 2-fold cross-validation (if any). <br>d. Construct a **confusion matrix** for your _most accurate_ model between the three estimators and two cross-fold options. <br> e. Which class in your dataset is most accurately predicted to have the correct label by the best classifier, and and which is most likely to be confused among one or more of the wrong classes?_(You can use a cell in a jupyter notebook file for this or a separate text/document file)._

**2 (Option I). (Machine Learning (Regression))** <br>a. Locate a non-proprietary, small-scale dataset _suitable for regression_ online.  There are countless sources and repositories than you can use in this task, but if you have trouble finding one, I recommend starting via Kaggle (https://www.kaggle.com/code/rtatman/datasets-for-regression-analysis/notebook).  Explain briefly what the dataset represents, what target variable you will be using, and what other features are present.  _You may want or need to apply preprocessing to your data to insure it can be used properly with the regression models_ (e.g. making every feature numeric through transformation or by dropping some)  <br> b. Train **three** distinct sklearn regression estimators for the chosen dataset and compare the results to see which one performs the best when using **10-fold cross-validation**, utilizing the R-Squared score to gauge performance.  Note that you should use two distinct regression models here (not just tweak underlying parameters).  A relatively complete listing of the available estimators can be found here (https://scikit-learn.org/stable/supervised_learning.html) -- but make sure you only use regression models!  Unless you have an inclination to do otherwise, I recommend using the model default parameters when available.<br>  c. Repeat part b utilizing the Mean Square Error to gauge performance.  _Briefly_ research the difference between the two metrics (MSE and R2), and explain in a paragraph or two i. the difference between them ii. when each one is the preferable metric to use. _(You can use a cell in a jupyter notebook file for this or a separate text/document file)._

**2 (Option II). (PRAW and Sentiment)** <br>*Note: you should feel free to propose an alternative task using PRAW of your own design to me via email if you have a specific one in mind.*
<br>a. Use PRAW to extract the 20 top submissions of all time from each of five related subreddits of your choice (ex: someone interested in sports subreddits might extract the top 20 posts from r/basketball, top 20 posts from r/football, etc.). 
<br>b. Use a sentiment analyzer (via Textblob or one of your choosing) to determine the positive sentiment of each and every top submission (all 100 in total), and store these in a variable of your choosing (data-frame, list of lists, etc.)  
c. Investigate how to use a Python [box-plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.boxplot.html).  After doing so, produce a box-plot (default parameters are fine) for the sentiment measures of each of your five subreddits.  Your presentation should look _something_ like the image at the bottom of this file, with your five chosen subreddits replacing the x-axis labels (Subreddit 1 .. Subreddit 5).
<br>d. Repeat steps a-c but use the 20 most controversial submissions of all time for each of your five subreddits.<br>e. Does anything surprise you about the distribution of sentiments, either with respect to individual subreddits, differences between the five subreddits, or the differences between _top_ and _controversial_ submissions?  Explain your answer in a few sentences.  _(You can use a cell in a jupyter notebook file for this or a separate text/document file)._
<br>![Box Plot Example](A4BPExample.png)
